In [1]:
import dask
import fsspec
import glob
# import requests
import datetime
import gc  

import matplotlib as mpl
import numpy as np
import pandas as pd
import xarray as xr

from collections import defaultdict
from matplotlib import pyplot as plt
from params import allnames

from dask.diagnostics import progress
from scipy.stats import norm
from tqdm.autonotebook import tqdm
import xml.etree.ElementTree as ET

from params import allnames
from params import homedir
from params import experiment_ids, years, table_ids, labels, variables, savepath

figdir = homedir + 'figures/'


import xml.etree.ElementTree as ET

from dask.diagnostics import progress
from scipy.stats import norm
from tqdm.autonotebook import tqdm

from params import allnames, experiment_ids, variables, years, table_ids, labels
from params import homedir, savepath

## ## style ## ##
xr.set_options(display_style='html')
plt.style.use('science.mplstyle')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['xtick.top']= False
mpl.rcParams['ytick.right']= False


/glade/scratch/mcowherd/ipykernel_152210/3062428812.py:19: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [1]:
import util

domains = ['d01','d02']
variables = ['snow', 't2', 'prec']
gcms = ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
        'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
        'canesm5','access-cm2','ec-earth3',]

variants = ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
            'r1i1p1f1','r1i1p1f1','r2i1p1f2',
            'r1i1p2f1','r5i1p1f1','r1i1p1f1',]
      

calendar = ['365_day','proleptic_gregorian','proleptic_gregorian',
            'proleptic_gregorian','365_day','360_day',
             '365_day','proleptic_gregorian', 'proleptic_gregorian',]

ssps = ['ssp370','ssp370','ssp370','ssp370',
        'ssp370','ssp370','ssp370','ssp370',
        'ssp370']

bcs = ['noBC', 'BC']

domains = ['d03', 'd04']
variables = ['snow', 't2', 'prec']
gcms = ['ec-earth3-veg',]
variants = ['r1i1p1f1',]
calendar = ['proleptic_gregorian']
ssps = ['ssp370']
bcs = ['noBC','BC']


datadir = '/glade/campaign/uwyo/wyom0112/postprocess/'
basedir = '/glade/campaign/uwyo/wyom0112/postprocess/'
savepath = '/glade/campaign/uwyo/wyom0112/berkeley/'

In [2]:
import os
## 
def wrfread(modeldir, gcm, exp, variant, domain, var):
    all_files = sorted(os.listdir(modeldir))
    read_files = []
    for ii in all_files:
        if (
            ii.startswith(var + ".")
            #and gcm in ii
            and variant in ii
            and domain in ii
            and exp in ii
        ):
            if domain in ii:
                read_files.append(os.path.join(modeldir, str(ii)))
    assert len(read_files) > 0, f"No matching files found in {modeldir}"

    del all_files

    data = xr.open_mfdataset(read_files, combine="by_coords")
    var_read = data.variables[var]

    dates = []
    for val in data["day"].data:
        try:
            dates.append(datetime.datetime.strptime(str(val)[0:-2], "%Y%m%d").date())
        except ValueError:
            dates.append(datetime.datetime(int(str(val)[0:4]), int(str(val)[4:6]), 28))


    var_read = xr.DataArray(var_read, dims=["day", "lat2d", "lon2d"])
    var_read["day"] = dates
    return var_read



In [3]:
import util

domains = ['d01','d02']
variables = ['snow', 't2', 'prec']
gcms = ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
        'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
        'canesm5','access-cm2','ec-earth3',]

variants = ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
            'r1i1p1f1','r1i1p1f1','r2i1p1f2',
            'r1i1p2f1','r5i1p1f1','r1i1p1f1',]
      

calendar = ['365_day','proleptic_gregorian','proleptic_gregorian',
            'proleptic_gregorian','365_day','360_day',
             '365_day','proleptic_gregorian', 'proleptic_gregorian',]

ssps = ['ssp370','ssp370','ssp370','ssp370',
        'ssp370','ssp370','ssp370','ssp370',
        'ssp370']

bcs = ['noBC', 'BC']

domains = ['d03', 'd04']
variables = ['snow', 't2', 'prec']
gcms = ['ec-earth3-veg',]
variants = ['r1i1p1f1',]
calendar = ['proleptic_gregorian']
ssps = ['ssp370']
bcs = ['noBC','BC']


datadir = '/glade/campaign/uwyo/wyom0112/postprocess/'
basedir = '/glade/campaign/uwyo/wyom0112/postprocess/'
savepath = '/glade/campaign/uwyo/wyom0112/berkeley/'

In [4]:

#basedir = '/global/cfs/cdirs/m4099/fate-of-snotel/wrfdata/'
model = None
n = 0
for domain in domains:
    for idx, gcm in enumerate(gcms[n:]):
        variant = variants[idx+n]

        for bc in bcs:
            if bc == 'BC':
                mod_historical = gcm +'_'+ variant + '_historical_bc'
                mod_future = gcm +'_' + variant+ '_ssp370_bc'
            if bc == 'noBC':
                mod_historical = gcm +'_'+ variant + '_historical'
                mod_future = gcm +'_' + variant+ '_ssp370'
            for var in variables:
                # historical
                model = mod_historical
                datadir = basedir + model + '/postprocess/' + domain + '/'
                date_start_pd, date_end_pd = [1980, 10, 1], [2014, 9, 30]  # 30 years, historical
                exp = "hist"
                var_wrf = wrfread(datadir, model, exp, variant, domain, var)
                var_wrf = util.screen_times_wrf(var_wrf, date_start_pd, date_end_pd)

                # future 
                date_start_pd, date_end_pd = [2013, 10, 1], [2100, 9, 30]
                model = mod_future
                exp = "ssp370"
                datadir = basedir + model + '/postprocess/' + domain + '/'
                var_wrf_ssp370 = wrfread(datadir, model, exp, variant, domain, var)
                var_wrf_ssp370 = util.screen_times_wrf(var_wrf_ssp370, date_start_pd, date_end_pd)
                wrfdata = [var_wrf, var_wrf_ssp370]
                ds_concat = xr.concat(wrfdata, dim = 'day').to_dataset(name = var)
                ds_concat = ds_concat.rename({'day':'time'})
                ds_concat['time'] = ds_concat['time'].astype('datetime64')
                try:
                    ds_concat.to_netcdf(f'{savepath}{var}_{gcms[idx+n]}_{domain}_{bc}.nc')
                    print('saved' + f'{savepath}{var}_{gcms[idx+n]}_{domain}_{bc}.nc')
                except:
                    print('already loaded')

saved/glade/campaign/uwyo/wyom0112/berkeley/snow_ec-earth3-veg_d03_noBC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/t2_ec-earth3-veg_d03_noBC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/prec_ec-earth3-veg_d03_noBC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/snow_ec-earth3-veg_d03_BC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/t2_ec-earth3-veg_d03_BC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/prec_ec-earth3-veg_d03_BC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/snow_ec-earth3-veg_d04_noBC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/t2_ec-earth3-veg_d04_noBC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/prec_ec-earth3-veg_d04_noBC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/snow_ec-earth3-veg_d04_BC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/t2_ec-earth3-veg_d04_BC.nc
saved/glade/campaign/uwyo/wyom0112/berkeley/prec_ec-earth3-veg_d04_BC.nc
